In [1]:
import os
import timm
import torch
from torch.utils.data import DataLoader
import numpy as np
import torchattacks
import torchvision.transforms as transforms
from torchvision import datasets
import albumentations as A
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from utils import training_loop,validation_loop,randAugment
from tqdm.notebook import tqdm
from torch.utils.data import Dataset
device= "cuda" if torch.cuda.is_available() else "cpu"
names = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]
num_classes=len(names)

In [2]:
import torchattacks
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

model1 = timm.create_model("resnext50_32x4d",pretrained=True,num_classes=num_classes).to(device) #trained on imagenet
model2 = timm.create_model("convnext_small",pretrained=True,num_classes=num_classes).to(device) # will be trained on cifar10

#ops = randAugment(N=2,M=5,p=0.5,mode="all",cut_out=True)
train_transform=transforms.Compose(
    [
        #transforms.RandAugment(num_ops=5,magnitude=2),

        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, padding=4),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ]
)

val_transform=transforms.Compose(
    [
                transforms.Resize(32),
                transforms.CenterCrop(32),
                transforms.ToTensor(),
                transforms.Normalize(mean, std),
    ]
)

def target_transform(label) :
     label_vector = torch.zeros((num_classes))
     label_vector[label]=1
     return label_vector

# class Cifar10SearchDataset(datasets.CIFAR10):
#     def __init__(self, root="~/data/cifar10", train=True, download=True, transform=None):
#         super().__init__(root=root, train=train, download=download, transform=transform)
#
#     def __getitem__(self, index):
#         image, label = self.data[index], self.targets[index]
#
#         if self.transform is not None:
#             transformed = self.transform(image=image)
#             image = transformed["image"]
#
#         return image, label
training_data = datasets.CIFAR10(
    root="/mnt/f/data",
    train=True,
    download=True,
    transform=train_transform,
    #target_transform=target_transform

)

test_data = datasets.CIFAR10(
    root="/mnt/f/data",
    train=False,
    download=True,
    transform=val_transform,
    #target_transform=target_transform

)





Files already downloaded and verified
Files already downloaded and verified


In [3]:
train_loader = DataLoader(
    training_data,
    num_workers=8,
    batch_size=256,
    shuffle=True
)
test_loader= DataLoader(
    test_data,
    num_workers=8,
    batch_size=256
)

In [4]:

metrics = {
    "accuracy" : [],
    "train_loss" : [],
    "val_loss" : [],
}
keep_training=False
criterion = lambda  pred, true : torch.nn.functional.cross_entropy(torch.softmax(pred,dim=1),true)
for model in  [model2,model1] :
    if os.path.exists(f"{model.__module__}.pt") :
        model.load_state_dict(torch.load(f"{model.__module__}.pt"))
        model = model.to(device)
        val_loss,results = validation_loop(model,test_loader,criterion,device,autocast=False)
        accuracy = (results[0].cpu().numpy()== np.argmax(torch.softmax(results[1].cpu(),dim=1).numpy(),axis=1)).mean()
        print(f"Model : {model.__module__} , Accuracy : {accuracy}")
    else :
        accuracy=0
    if keep_training and accuracy<0.85 :
        optimizer = torch.optim.AdamW(model.parameters(),lr=1e-4,weight_decay=1e-3)


        scaler = torch.cuda.amp.GradScaler()
        max_epoch = 30
        scheduler=torch.optim.lr_scheduler.OneCycleLR(max_lr=1e-4,optimizer=optimizer,epochs=max_epoch,steps_per_epoch=len(train_loader))

        for epoch in (pbar:=tqdm(range(0,max_epoch),position=0,leave=True)) :
            loss = training_loop(model,tqdm(train_loader,position=1,leave=False),optimizer=optimizer,criterion=criterion,device=device,scaler=scaler,clip_norm=0,autocast=False,scheduler=scheduler,epoch=max_epoch)

            val_loss,results = validation_loop(model,test_loader,criterion,device,autocast=False)
            accuracy = (results[0].cpu().numpy()== np.argmax(torch.softmax(results[1].cpu(),dim=1).numpy(),axis=1)).mean()
            pbar.set_description(f"Training loss : {loss/len(train_loader)} ,  validation loss : {val_loss/len(test_loader)} , accuracy : {accuracy}")
            metrics["accuracy"].append(accuracy)
            metrics["train_loss"].append(loss/len(train_loader))
            metrics["val_loss"].append(val_loss/len(test_loader))


        torch.save(model.state_dict(), f"{model.__module__}.pt")
        model.to("cpu")



Model : timm.models.convnext , Accuracy : 0.9147
Model : timm.models.resnet , Accuracy : 0.8648


In [8]:

# If, images are normalized:
# atk.set_normalization_used(mean=[...], std=[...])
optimizer = torch.optim.Adam(model1.parameters(),lr=1e-3)
train_transform=transforms.Compose(
    [
        #transforms.RandAugment(num_ops=5,magnitude=2),

        #transforms.RandomHorizontalFlip(),
        #transforms.RandomCrop(32, padding=4),
        #transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ]
)
training_data = datasets.CIFAR10(
    root="/mnt/f/data",
    train=True,
    download=True,
    transform=train_transform,
    #target_transform=target_transform

)

train_loader = DataLoader(
    training_data,
    num_workers=4,
    batch_size=4,
    shuffle=True
)

metrics = {
    "sensitivity" : [],
    "loss" : []
}
model1 = model1.to(device)
model2 = model2.to(device)
#train_loader.batch_size=128
eps=0.08
atk = torchattacks.PGD(model1, eps=eps*255, alpha=2/255, steps=10)
#atk.loss = torch.nn.MSELoss()

for epoch in (pbar:=tqdm(range(0,50),leave=True,position=0)) :

    for images,labels in tqdm(train_loader,position=1,leave=False) :


        images,labels = images.to(device),labels.to(device)
        adv_images = atk(images, labels)
        adv_images = adv_images.to(device)

        logit_model1_adv_img    = model1(adv_images)
        logit_model2_img        = model2(images)
        logit_model2_adv_img    = model2(adv_images)


        loss = torch.mean((logit_model2_adv_img-logit_model1_adv_img)**2)
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        sensitivity = torch.mean(torch.abs(logit_model2_img-logit_model2_adv_img))
    pbar.set_description(f"Sensitivity : {sensitivity} ,  loss : {loss}")
        metrics["sensitivity"].append(sensitivity)
        metrics["loss"].append(loss)

        #adv_pred = torch.softmax(model1(adv_images),dim=1)




Files already downloaded and verified


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/12500 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 6.00 GiB total capacity; 5.25 GiB already allocated; 0 bytes free; 5.29 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF